# Dataset Generation

This script allows for fast generation of a dataset of cloud and shadow masks. The masks can then be mixed with any new image to generate a version with clouds and shadows (see an example of posterior mixing at the very bottom!)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import rasterio as rio
from pathlib import Path
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import imageio

from src import *


# visualization
plt.rcParams["figure.figsize"] = (4,4)
GPU_IDX = 0
rgb_img = imageio.imread('imgs/rgb_example.png')[...,:3]/255
const_img=np.ones_like(rgb_img)*0.5

In [ ]:
# dataset params
N=20 # image count
shape=(256,256) # image shape
data_dir=Path("./data/cloud_dataset")

## Group 1: Thick Clouds

In [ ]:
out_dir=(data_dir / "thick")
out_dir.mkdir(parents=True, exist_ok=True)

display=True

for idx in range(N):

    cl, cmask, smask = add_cloud_and_shadow(rgb_img,
                                            return_cloud=True
                                           )

    if display:
        plt.figure()
        plt.subplot(1,3,1)
        plt.imshow(cl)
        plt.axis('off')
        plt.subplot(1,3,2)
        plt.imshow(cmask)
        plt.axis('off')
        plt.subplot(1,3,3)
        plt.imshow(smask)
        plt.axis('off')
        plt.tight_layout()
    
    # save images
    with open(out_dir / "cloud_{}.npy".format(idx), 'wb') as f:
        np.save(f,cmask)
    with open(out_dir / "shadow_{}.npy".format(idx), 'wb') as f:
        np.save(f,smask)

## Group 2: Local

In [ ]:
out_dir=(data_dir / "local")
out_dir.mkdir(parents=True, exist_ok=True)

display=True

for idx in range(N):

    cl, cmask, smask = add_cloud_and_shadow(rgb_img,
                                            locality_degree=3,
                                            return_cloud=True
                                           )

    if display:
        plt.figure()
        plt.subplot(1,3,1)
        plt.imshow(cl)
        plt.axis('off')
        plt.subplot(1,3,2)
        plt.imshow(cmask)
        plt.axis('off')
        plt.subplot(1,3,3)
        plt.imshow(smask)
        plt.axis('off')
        plt.tight_layout()
    
    # save images
    with open(out_dir / "cloud_{}.npy".format(idx), 'wb') as f:
        np.save(f,cmask)
    with open(out_dir / "shadow_{}.npy".format(idx), 'wb') as f:
        np.save(f,smask)

## Group 3: Haze

In [ ]:
out_dir=(data_dir / "thin")
out_dir.mkdir(parents=True, exist_ok=True)

display=True

for idx in range(N):

    cl, cmask = add_cloud(rgb_img,
                                 min_lvl=0.4,
                                 max_lvl=0.6,
                                 decay_factor=1.5,
                                 return_cloud=True
                                )
    
    smask=torch.zeros_like(cmask)

    if display:
        plt.figure()
        plt.subplot(1,3,1)
        plt.imshow(cl)
        plt.axis('off')
        plt.subplot(1,3,2)
        plt.imshow(cmask)
        plt.axis('off')
        plt.subplot(1,3,3)
        plt.imshow(smask)
        plt.axis('off')
        plt.tight_layout()
    
    # save images
    with open(out_dir / "cloud_{}.npy".format(idx), 'wb') as f:
        np.save(f,cmask)
    with open(out_dir / "shadow_{}.npy".format(idx), 'wb') as f:
        np.save(f,smask)

# Dataset Usage
If we have an clear image like the `rgb_image`, it can be mixed with any new cloud using the `mix()` function in `SatelliteCloudGenerator` (make sure you import the package).

In [ ]:
# our new image
rgb_img = torch.from_numpy(imageio.imread('imgs/rgb_example.png')[...,:3]/255)


# a cloud from our dataset
with open('./data/cloud_dataset/local/cloud_1.npy', 'rb') as f:
    cmask = torch.from_numpy(np.load(f))
    
# a shadow from our dataset
with open('./data/cloud_dataset/thick/shadow_1.npy', 'rb') as f:
    smask = torch.from_numpy(np.load(f))
    
# apply mixing
cloudy_img = mix(rgb_img, cmask)

In [ ]:
plt.figure(figsize=(10,10))

plt.subplot(1,3,1)
plt.imshow(rgb_img)
plt.title('Example Image')
plt.subplot(1,3,2)
plt.imshow(cmask)
plt.title('Cloud from Dataset')
plt.subplot(1,3,3)
plt.imshow(cloudy_img)
plt.title('Result')

In [ ]:
cloudy_shadowy_img = mix(rgb_img, cmask, shadow=smask)

In [ ]:
plt.figure(figsize=(10,10))

plt.subplot(1,4,1)
plt.imshow(rgb_img)
plt.title('Example Image')
plt.subplot(1,4,2)
plt.imshow(cmask)
plt.title('Cloud from Dataset')
plt.subplot(1,4,3)
plt.imshow(smask)
plt.title('Shadow from Dataset')
plt.subplot(1,4,4)
plt.imshow(cloudy_shadowy_img)
plt.title('Result')